# Шифрование обратимой матрицей

Нам нужно защитить данные клиентов. Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Оглавление:
* [1. Загрузка данных](#1)
* [2. Умножение матриц](#2)
* [3. Алгоритм преобразования](#3)
* [4. Проверка алгоритма](#4)

## 1. Загрузка данных <a class="anchor" id="1"></a>

Импортируем библиотеки:

In [1]:
# <импорт библиотеки pandas>
import pandas as pd

# <импорт библиотеки sklearn>
import sklearn

# <Отключение предупреждений>
import warnings
warnings.filterwarnings('ignore')

# <импорт библиотеки numpy>
import numpy as np

Прочитаем файл с данными:

In [2]:
# <чтение файла с данными с сохранением в data_full>
data = pd.read_csv('/datasets/insurance.csv')

Рассмотрим информацию по датафрейму и его первые 5 строк:

In [3]:
# <рассмотрим датафрейм data>
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


В датафрейме присутствуют:

* Признаки - пол, возраст и зарплата застрахованного, количество членов его семьи.
* Целевой признак - количество страховых выплат клиенту за последние 5 лет.

### Вывод

Данные загружены, пропусков нет.

## 2. Умножение матриц <a class="anchor" id="2"></a>

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

### Умножение признаков на обратимую матрицу, изменится ли качество лин. регрессии?

**Ответ:** Не изменится.

**Обоснование:** 

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Вместо X подставим XP, где P - обратимая квадратная матрица со случайными числами.

$$
Z = XP
$$

$$
w(Z) = ((XP)^T XP)^{-1} (XP)^T y
$$

Используем формулу:

$$
(AB)^T = B^T A^T
$$

$$
w(Z) = (P^T X^T XP)^{-1} P^T X^T y
$$

Используем формулу(она работает для квадратных обратимых матриц):

$$
(AB)^{-1} = B^{-1} A^{-1}
$$

Обращаем внимание что 

$$
P^T, X^T X, P
$$

являются квадратными. Преобразуем то что в скобке.

$$
w(Z) = P^{-1} (P^T X^T X)^{-1} P^T X^T y
$$


$$
w(Z) = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Помним также что умножение матрицы на обратную матрицу равно единичной матрице, а умножение любой матрицы на единичную равно этой самой матрице.

$$
w(Z) = P^{-1} (X^T X)^{-1} X^T y
$$

Подставим *w(Z)* в формулу предсказаний.

$$
a(Z) = X P P^{-1} (X^T X)^{-1} X^T y
$$

$$
a(Z) = X (X^T X)^{-1} X^T y
$$

Результат не изменился, что и требовалось доказать.

### Вывод

Мы доказали с помощью матричных преобразований что предсказания линейной регресии при умножении матрицы признаков на другую матрицу (с размерностью 4x4, так как нам требуется в итоге матрица 5000x4) не изменятся.

## 3. Алгоритм преобразования <a class="anchor" id="3"></a>

**Алгоритм**

Умножим матрицу признаков на матрицу 4x4, так как количество столбцов матрицы A должно быть равно количеству строк матрицы B при умножении, а матрица полученная после умножения будет иметь количество строк матрицы A и количество столбцов матрицы B.

Саму матрицу B сгенерируем с помощью метода библиотеки numpy np.random.normal(), где первый аргумент среднее нормального распределения, второй стандартное отклонение, третий - размерность матрицы.

**Обоснование**

Создадим переменные для признаков и целевого признака.

In [4]:
# <создадим переменные для признаков и целевого признака>
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

Создадим обратимую квадратную матрицу со случайными числами размерностью 4x4, умножим наши признаки на нее и полученную матрицу сделаем датафреймом.

In [5]:
random_matrix = np.random.normal(3, 2.5, size=(4, 4))
features_transformed = pd.DataFrame(features.values @ random_matrix, index = features.index, columns = features.columns)
features_transformed

,Пол,Возраст,Зарплата,Члены семьи
0,-2931.223114,-40767.298786,214661.147473,396185.897650
1,-2285.329658,-31145.350823,164494.704660,303607.453210
2,-1271.929294,-17190.956691,90910.902879,167801.410786
3,-2433.037177,-34354.064292,180444.821260,333012.427717
4,-1557.149699,-21413.968341,112969.432520,208510.967072
...,...,...,...,...
4995,-2108.524995,-29350.567090,154507.002118,285151.206560
4996,-3076.373025,-43124.918671,226757.002794,418500.145789
4997,-1985.470775,-27910.171452,146701.390723,270738.633091
4998,-1919.107033,-26904.648011,141519.579876,261172.123637


Проверим что матрица обратима.

In [6]:
np.linalg.inv(random_matrix)

array([[ 0.29405648, -0.0275912 , -0.03065751,  0.07232219],
       [ 0.11105713,  0.13867048, -0.11552685,  0.04628663],
       [-0.01643105, -0.04153058, -0.01753099,  0.24213429],
       [ 0.02250851,  0.03667053,  0.12257791, -0.12589255]])

### Вывод

Мы преобразовали признаки и в следующем шаге проверим что метрика качества линейной регресии не изменится после преобразований.

## 4. Проверка алгоритма <a class="anchor" id="4"></a>

Импортируем библиотеки:

In [7]:
# <Импортируем метод линейной регрессии>
from sklearn.linear_model import LinearRegression

# <Импортируем коэффициент детерминации>
from sklearn.metrics import r2_score

Проверим качество линейной регресии на непреобразованных данных:

In [8]:
# <Создадим модель лин. регрессии>
model = LinearRegression().fit(features, target)

# <обучаем модель> 
predictions = model.predict(features)

# <метрика качества коэффициент детерминации>
print('R2:','{:.4f}'.format(r2_score(target,predictions)))

R2: 0.4249


Проверим качество линейной регресии на преобразованных данных:

In [9]:
# <Создадим модель лин. регрессии>
model = LinearRegression().fit(features_transformed, target)

# <обучаем модель> 
predictions = model.predict(features_transformed)

# <метрика качества коэффициент детерминации>
print('R2:','{:.4f}'.format(r2_score(target,predictions)))

R2: 0.4249


Качество не отличается. 

### Вывод

Мы удостоверились что качество линейной регресии не меняется при умножении признаков на обратимую матрицу со случайными числами.